In [ ]:
# Load the python package
import os
# import dynetan
from dynetan.toolkit import *
from dynetan.viz import *
from dynetan.proctraj import *
from dynetan.gencor import *
from dynetan.contact import *
from dynetan.datastorage import *

#from numpy.linalg import norm
from itertools import islice
from scipy import stats

import pandas as pd
import numpy as np
import scipy as sp

In [ ]:
dnad = DNAdata()

# Load network data obtained by Dynetan module 

In [ ]:
dataDir = "......" ## Directory where the first notebook data is stored

# Path where results will be written (you may want plots and data files in a new location)
workDir = "....."

fileNameRoot = "Evo0MM_network" ## from first notebook output [look for the prefix of .hf files]
fullPathRoot = os.path.join(dataDir, fileNameRoot)

# Define the segID of the Ligand being studied.
ligandSegID = "SYST"
dnad.loadFromFile(fullPathRoot)
dcdVizFile = fullPathRoot + "_reducedTraj.dcd"
pdbVizFile = fullPathRoot + "_reducedTraj.pdb"

workUviz = mda.Universe(pdbVizFile, dcdVizFile)
# We add this to the object for ease of access.
dnad.nodesAtmSel = workUviz.atoms[ dnad.nodesIxArray ]
print(dnad.nodesAtmSel)

A function to calculate pathlengths in terms of edge betweenness:

## Calculate inter-domain betweennesses which is a more robust data than intercommunity betweenness data. 

In [ ]:

# a function to assign nodes to a specific domain
def search(lst, value):
    for i in range(len(lst)):
        if value in lst[i]:
            return i


# define a list of domain residue ranges
domain = [list(range(167,307)), list(range(494, 710)), list(range(762, 906)), list(range(1451, 1470)), list(range(1362, 1381))] 



def interdomain(domain_list, output_prefix):
    
    # create a 2d np.array to fill domain betweenness
    domain_bet = np.zeros(shape=(len(domain), len(domain)))
    
    for winIndx in range(dnad.numWinds):
        
        bet = np.zeros(shape=(dnad.corrMatAll[winIndx,:,:].shape[0], dnad.corrMatAll[winIndx,:,:].shape[1]))
        for pair,btw in dnad.btws[winIndx].items():
            bet[pair[0], pair[1]] = btw

        normBetMat = copy.deepcopy(bet)
        normBetMat /= normBetMat.max()
        
        for pair,btw in dnad.btws[winIndx].items():
            
           ## A running matrix to fill out domain betweenness
            running_dom = np.zeros(shape=(len(domain), len(domain)))
            node1 = pair[0]
            node2 = pair[1]

          # if the nodes doesn't belong to the same domain, calculate the betweenness between them
            max_list = [item for sublist in domain_list for item in sublist]
            if (node1 < max(max_list)) and (node2 < max(max_list)):
                if search(domain,node1) != search(domain,node2):
                    if (normBetMat[ node1, node2] != 0) & (search(domain,node1) != None) & (search(domain,node2) != None):
                        running_dom[search(domain,node1), search(domain,node2)] = normBetMat[ node1, node2]
                        running_dom[search(domain,node2), search(domain,node1)] = normBetMat[ node1, node2]
                        domain_bet = np.add(domain_bet, running_dom) # updating after each frame

    print(domain_bet)
    domain_bet_round = np.round_(domain_bet, decimals = 3)
    np.save(os.path.join(workDir, output_prefix+".npy"), domain_bet_round)
    np.savetxt(os.path.join(workDir, output_prefix+".csv"), domain_bet_round, fmt='%.3f', delimiter=",")

interdomain(domain, 'Evo_4rep_InterDomain_betweenness_test')

# This section reads the interdomain betweenness matrices and calculate the difference in interdomain betweennesses of 2 comparable systems...

In [ ]:
## differential domain betweenness #######
def diff_domain_between(filename1, filename2, outputfilename): #all csv format files e.g.: filename1= interdomain_betweenness_s1.csv
    
    import numpy as np
    from numpy import genfromtxt
    s1 = genfromtxt(os.path.join(workDir, filename1), delimiter=',') # SYS-1
    s2 = genfromtxt(os.path.join(workDir, filename2), delimiter=',') # SYS-2
    
    t1 =  np.zeros(shape=(s1.shape[0], s1.shape[1]))

    for i in range(s1.shape[0]):
        for j in range(s1.shape[1]):
            t1[i,j] = s1[i,j] - s2[i,j]
    print("system-1:", s1)
    print("system-2:", s2)
    print("Difference:", t1)
    np.savetxt(os.path.join(workDir, outputfilename), t1, fmt='%.3f', delimiter=",")
    

In [ ]:
diff_domain_between("sys1_betweenness_file", "sys2_betweenness_file", "SYS1-SYS2_output")

## For read graph and visulaize network

In [ ]:
import numpy as np
import pandas as pd

Load the differential interdomain betweenness data (two system comparison):

In [ ]:
interdomain_bet = np.genfromtxt("...InterDomain_betweenness.csv", delimiter=",")


print(interdomain_bet)
print(interdomain_bet.shape)

In [ ]:
b = np.triu(interdomain_bet)
print(b)

In [ ]:
node_names = ['REC2', 'REC3', 'HNH', 'TS', 'guideRNA'] ## Give names to your domains

In [ ]:
import matplotlib.pyplot as plt
# %matplotlib inline
# fig, ax = plt.subplots()
# ig.plot(g, target=ax)
import igraph as ig 
from igraph import *

# get the row, col indices of the non-zero elements in your adjacency matrix
conn_indices = np.where(b)

# get the weights corresponding to these indices
weights = b[conn_indices]

# a sequence of (i, j) tuples, each corresponding to an edge from i -> j
edges = zip(*conn_indices)

# initialize the graph from the edge sequence
G = ig.Graph(edges=edges, directed=False)

color_list = []
for i in weights:
    if i < 0:
        color_list.append('red')
    else:
        color_list.append('cyan')
# assign node names and weights to be attributes of the vertices and edges
# respectively
G.vs['label'] = node_names
G.es['color'] = color_list
G.es['weight'] = np.absolute(weights)


# I will also assign the weights to the 'width' attribute of the edges. this
# means that igraph.plot will set the line thicknesses according to the edge
# weights
G.es['width'] = np.absolute(weights)



# plot the graph, just for fun
# igraph.plot(G, layout="rt", labels=True, margin=80, target=ax)
layout = G.layout("circle")
visual_style = {}
visual_style["vertex_size"] = 70
visual_style["vertex_color"] = "wheat"
visual_style["labels"] = True
visual_style["margin"] = 80
visual_style["layout"] = layout

# ig.plot(G, vertex_size=70, vertex_color="wheat", layout=layout, labels=True, margin=80)
ig.plot(G,".....png", **visual_style)